In [1]:
import numpy as np
import pandas as pd

In [2]:
LoadPath_SubwayStation_Schedule = "../raw_data/selected/부산교통공사_부산도시철도 운행 정보_20230710.csv"
SavePath= "../trimmed_data"
TARGET_COLUMNS = ["요일구분","열차번호" ,"운행구간정거장","정거장도착시각","정거장출발시각"]
TARGET_COLUMNS_ENG = ["DayType","TrainCode","StationList", "ArriveSchedule","DepartSchedule"]
# "운행구간기점명", "운행구간종점명",
# "StartStationName", "EndStationName",

# `SubwayUniStation-Schedule.ipynb`
> `배차간역` : 한 역에서 어떤 열차가 지나가고 다음 열차가 올 때까지의 시간
- 한 역에서의 `배차간격` 데이터

## 1. 데이터 불러오기

In [3]:
# RawData
db = pd.read_csv(LoadPath_SubwayStation_Schedule,encoding="EUC-KR")
db.head(3)

,열차번호,노선번호,노선명,운행구간기점명,운행구간종점명,운행유형,요일구분,운행구간정거장,정거장도착시각,정거장출발시각,운행속도,운영기관전화번호,데이터기준일자
0,1001,S2601,부산 도시철도 1호선,노포,다대포해수욕장,일반,평일,001-노포+002-범어사+003-남산+004-두실+005-구서+006-장전+007...,001-05:08+002-05:09+003-05:11+004-05:13+005-05...,001-05:08+002-05:10+003-05:12+004-05:13+005-05...,39.6km/h,1544-5005,2023-07-11
1,1003,S2601,부산 도시철도 1호선,노포,다대포해수욕장,일반,평일,001-노포+002-범어사+003-남산+004-두실+005-구서+006-장전+007...,001-05:18+002-05:20+003-05:22+004-05:23+005-05...,001-05:18+002-05:20+003-05:22+004-05:24+005-05...,39.6km/h,1544-5005,2023-07-11
2,1005,S2601,부산 도시철도 1호선,노포,다대포해수욕장,일반,평일,001-노포+002-범어사+003-남산+004-두실+005-구서+006-장전+007...,001-05:29+002-05:31+003-05:33+004-05:34+005-05...,001-05:29+002-05:31+003-05:33+004-05:35+005-05...,39.6km/h,1544-5005,2023-07-11


## 2. 데이터 처리 및 임베딩

### 2.1 방향 임베딩

In [4]:
# 노선의 방향을 Embedding
directionVector = db["운행구간기점명"] + db["운행구간종점명"]
directionVector_eigen = list(set((directionVector).to_list()))
directionVector_eigen = [
    "다대포해수욕장노포","노포다대포해수욕장",
    "신평노포","노포신평",
    "장산호포","호포장산",
    "장산양산","양산장산",
    "전포양산","양산전포",
    "수영대저","대저수영",
    "미남안평","안평미남",
    "신평다대포해수욕장",
    "호포양산",
    "광안양산",
    "구명장산",
    "장산광안",
]

# Check Embedding
directionVector_index = [ directionVector_eigen.index(direct) for direct in directionVector ]
print(f"EMBEDING TARGET :\n{directionVector_eigen}\n{set(directionVector_index)}")

EMBEDING TARGET :
['다대포해수욕장노포', '노포다대포해수욕장', '신평노포', '노포신평', '장산호포', '호포장산', '장산양산', '양산장산', '전포양산', '양산전포', '수영대저', '대저수영', '미남안평', '안평미남', '신평다대포해수욕장', '호포양산', '광안양산', '구명장산', '장산광안']
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}


In [5]:
STATION_SCHEDULE = db.loc[:, TARGET_COLUMNS]
STATION_SCHEDULE = STATION_SCHEDULE.rename(
    columns={idx: new_name for idx, new_name in zip(TARGET_COLUMNS, TARGET_COLUMNS_ENG)}
)

# Insert Direction Vector
STATION_SCHEDULE.insert(loc=2,column="DirectionVector",value=pd.Series(directionVector_index))

# Save
with open(f"{SavePath}/SubwayStation-Schedule.csv",mode="w",encoding="UTF-8") as f:
    f.write(STATION_SCHEDULE.to_csv(index=False))
STATION_SCHEDULE.head(3)

,DayType,TrainCode,DirectionVector,StationList,ArriveSchedule,DepartSchedule
0,평일,1001,1,001-노포+002-범어사+003-남산+004-두실+005-구서+006-장전+007...,001-05:08+002-05:09+003-05:11+004-05:13+005-05...,001-05:08+002-05:10+003-05:12+004-05:13+005-05...
1,평일,1003,1,001-노포+002-범어사+003-남산+004-두실+005-구서+006-장전+007...,001-05:18+002-05:20+003-05:22+004-05:23+005-05...,001-05:18+002-05:20+003-05:22+004-05:24+005-05...
2,평일,1005,1,001-노포+002-범어사+003-남산+004-두실+005-구서+006-장전+007...,001-05:29+002-05:31+003-05:33+004-05:34+005-05...,001-05:29+002-05:31+003-05:33+004-05:35+005-05...


### 2.2 시간 데이터 처리 및 역명 처리

In [6]:
# 도착시간을 minute으로 mapping 하기위한 함수 정의
def hhmm_to_minute(hhmm: str):
    """
    hhmm_to_minute String Time Format(hh:mm) to Int Minute Time Format for Calculate Difference

    Args:
        hhmm (string): hh:mm format Time String

    Returns:
        int: Minute Time
    """
    HM = hhmm.split(":")
    if "" in HM:
        return np.NaN
    H, M = int(HM[0]), int(HM[1])
    minutes = H * 60 + M if H != 0 else 24 * 60 + M
    return minutes

# trimming 1
trim_target = TARGET_COLUMNS_ENG[2:5]
trimmed_data = [
    STATION_SCHEDULE[trim_target].loc[:, trim_target[0]]
    .apply(lambda _x: _x.split("+"))
    .apply(lambda _x: [e.split("-")[-1] for e in _x]),
    STATION_SCHEDULE[trim_target].loc[:, trim_target[1]]
    .apply(lambda _x: _x.split("+"))
    .apply(lambda _x: [e.split("-")[-1] for e in _x])
    .apply(lambda _x: [hhmm_to_minute(e) for e in _x]),
    STATION_SCHEDULE[trim_target].loc[:, trim_target[2]]
    .apply(lambda _x: _x.split("+"))
    .apply(lambda _x: [e.split("-")[-1] for e in _x])
    .apply(lambda _x: [hhmm_to_minute(e) for e in _x]),
]

# Update to trimmed data
for target_feature, data in zip(trim_target,trimmed_data):
    STATION_SCHEDULE[target_feature] = data

# trimming 2 - 노선:INT 추가
STATION_SCHEDULE.insert(loc=2,column="LineNumber",value=db["노선명"].apply(lambda _x: _x.split(" ")[-1].replace("호선","")))

#Check
STATION_SCHEDULE.head(3)

,DayType,TrainCode,LineNumber,DirectionVector,StationList,ArriveSchedule,DepartSchedule
0,평일,1001,1,1,"[노포, 범어사, 남산, 두실, 구서, 장전, 부산대, 온천장, 명륜, 동래, 교대...","[308, 309, 311, 313, 315, 317, 319, 321, 323, ...","[308, 310, 312, 313, 315, 317, 319, 321, 323, ..."
1,평일,1003,1,1,"[노포, 범어사, 남산, 두실, 구서, 장전, 부산대, 온천장, 명륜, 동래, 교대...","[318, 320, 322, 323, 325, 327, 329, 331, 333, ...","[318, 320, 322, 324, 326, 328, 329, 331, 333, ..."
2,평일,1005,1,1,"[노포, 범어사, 남산, 두실, 구서, 장전, 부산대, 온천장, 명륜, 동래, 교대...","[329, 331, 333, 334, 336, 338, 340, 342, 344, ...","[329, 331, 333, 335, 337, 339, 340, 342, 344, ..."


## 3 요일 별로 데이터 분리하기 

In [7]:
# 요일 Embedding
dayType_eigen = list(set((db["요일구분"]).to_list()))
print(dayType_eigen)

# 요일별로 저장
for day in dayType_eigen:
    with open(f"{SavePath}/SubwayStation-Schedule-{day}.json",mode="w",encoding="UTF-8") as f:
        f.write(STATION_SCHEDULE[db["요일구분"] == day].iloc[:,1:].to_json(index=False))

['토요일', '평일', '일요일', '공휴일']
